## AI or not AI that is the question
### Сравнение разметки двух экспертов (людей) при классификации изображений, созданных искусственным интеллектом (AI) или человеком (notAI). Анализ согласованности оценок экспертов и выявление потенциальных различий в их восприятии создания изображений.

ТЗ и сбор обратной связи по разметке данных находились в гугл форме (https://forms.gle/ZCuRHBRrucqDdM3V6)

In [14]:
import json
import pandas as pd
from sklearn.metrics import cohen_kappa_score, precision_score, recall_score, f1_score

In [2]:
# вытаскиваем результаты первой разметки
with open('export-result.ndjson', 'r') as json_file:
    data = [json.loads(line) for line in json_file]

external_ids = [] # тут лежат картинки
names = [] # тут лежат метки 

for item in data:
    external_ids.append(item['data_row']['external_id'])
    name = item['projects']['clp7hwgav08xb07z28maj52g0']['labels'][0]['annotations']['classifications'][0]['radio_answer']['name']
    names.append(name)

df = pd.DataFrame({'id': external_ids, 'label_1': names})
df

,id,label_1
0,0d2886762ebdde8d96364f88635487cb.jpg,notAI
1,0ff39f588a9d40f14d7df64d3596dc8c.jpg,notAI
2,0mN0djg-Z3s.jpg,notAI
3,1a8a12d83f65bb5dd75ceffe225b0da7.jpg,notAI
4,1bc894c745b58d12594b7c523246836c.jpg,notAI
...,...,...
197,AnyConv.com__fabef693-9fe1-4245-bf88-424b00d43...,AI
198,AnyConv.com__fae26a77-7c8e-46c7-a807-78f83a9a8...,AI
199,AnyConv.com__fb0fc2a2-7fbe-4a50-b6dd-dbde4e5ba...,AI
200,AnyConv.com__fda1a311-4df4-496f-8758-b5a0821e5...,AI


In [3]:
# добавляем столбец с правильными ответами
df.insert(1, 'AI_notAI', [0]*101 + [1]*101)

In [4]:
# переводим разметку в бинарный вид
df['label_1binary'] = df['label_1'].apply(lambda x: 1 if x == 'AI' else 0)

In [5]:
df

,id,AI_notAI,label_1,label_1binary
0,0d2886762ebdde8d96364f88635487cb.jpg,0,notAI,0
1,0ff39f588a9d40f14d7df64d3596dc8c.jpg,0,notAI,0
2,0mN0djg-Z3s.jpg,0,notAI,0
3,1a8a12d83f65bb5dd75ceffe225b0da7.jpg,0,notAI,0
4,1bc894c745b58d12594b7c523246836c.jpg,0,notAI,0
...,...,...,...,...
197,AnyConv.com__fabef693-9fe1-4245-bf88-424b00d43...,1,AI,1
198,AnyConv.com__fae26a77-7c8e-46c7-a807-78f83a9a8...,1,AI,1
199,AnyConv.com__fb0fc2a2-7fbe-4a50-b6dd-dbde4e5ba...,1,AI,1
200,AnyConv.com__fda1a311-4df4-496f-8758-b5a0821e5...,1,AI,1


In [6]:
print(f"Всего ошибок у первого разметчика: {(df['label_1binary'] != df['AI_notAI']).sum()}")

Всего ошибок у первого разметчика: 7


In [7]:
# вытаскиваем результаты второй разметки
with open('export-result2.ndjson', 'r') as json_file:
    data = [json.loads(line) for line in json_file]

external_ids = []
names = []

for item in data:
    external_ids.append(item['data_row']['external_id'])
    name = item['projects']['clp8brken035q07w5a7utabm6']['labels'][0]['annotations']['classifications'][0]['radio_answer']['name']
    names.append(name)

df_new = pd.DataFrame({'id': external_ids, 'label_2': names})
df_new

,id,label_2
0,AnyConv.com__00fbeab4-fa8f-4d5c-bea2-c805b3270...,AI
1,AnyConv.com__0bd0d329-fece-4589-9ab1-24d7b6eab...,notAI
2,AnyConv.com__0d5d8f19-87f0-4e0c-815d-01b84d14f...,AI
3,AnyConv.com__0f07aa6b-a048-4d60-9a15-520126615...,notAI
4,AnyConv.com__1b644565-c1b8-4b1a-afbc-c413518d2...,AI
...,...,...
197,xU-sHui3vPI.jpg,notAI
198,Z9_9A8INgNQ.jpg,notAI
199,Z9qB7TItNnQ.jpg,notAI
200,15fffef0-ae9d-4e55-b211-ed6c1397b774.jpg,notAI


In [8]:
# соединяем результаты разметок
result_df = pd.merge(df, df_new, on='id', how='inner')

In [9]:
# переводим вторую разметку в бинарный вид
result_df['label_2binary'] = result_df['label_2'].apply(lambda x: 1 if x == 'AI' else 0)

In [10]:
result_df

,id,AI_notAI,label_1,label_1binary,label_2,label_2binary
0,0d2886762ebdde8d96364f88635487cb.jpg,0,notAI,0,notAI,0
1,0ff39f588a9d40f14d7df64d3596dc8c.jpg,0,notAI,0,notAI,0
2,0mN0djg-Z3s.jpg,0,notAI,0,notAI,0
3,1a8a12d83f65bb5dd75ceffe225b0da7.jpg,0,notAI,0,notAI,0
4,1bc894c745b58d12594b7c523246836c.jpg,0,notAI,0,AI,1
...,...,...,...,...,...,...
197,AnyConv.com__fabef693-9fe1-4245-bf88-424b00d43...,1,AI,1,AI,1
198,AnyConv.com__fae26a77-7c8e-46c7-a807-78f83a9a8...,1,AI,1,AI,1
199,AnyConv.com__fb0fc2a2-7fbe-4a50-b6dd-dbde4e5ba...,1,AI,1,notAI,0
200,AnyConv.com__fda1a311-4df4-496f-8758-b5a0821e5...,1,AI,1,AI,1


In [11]:
print(f"Всего ошибок у второго разметчика: {(result_df['label_2binary'] != result_df['AI_notAI']).sum()}")

Всего ошибок у второго разметчика: 83


In [12]:
# Применяем меру согласованности Каппа Коэна 

# Рассчитываем Каппа Коэна между разметчиками 1 и 2
kappa_1_2 = cohen_kappa_score(result_df['label_1binary'], result_df['label_2binary'])

# Рассчитываем Каппа Коэна между фактическими метками и разметчиком 1
kappa_actual_1 = cohen_kappa_score(result_df['AI_notAI'], result_df['label_1binary'])

# Рассчитываем Каппа Коэна между фактическими метками и разметчиком 2
kappa_actual_2 = cohen_kappa_score(result_df['AI_notAI'], result_df['label_2binary'])

In [13]:
print(f"Kappa between annotators 1 and 2: {kappa_1_2}")
print(f"Kappa between actual labels and annotator 1: {kappa_actual_1}")
print(f"Kappa between actual labels and annotator 2: {kappa_actual_2}")

Kappa between annotators 1 and 2: 0.2104748876294703
Kappa between actual labels and annotator 1: 0.9306930693069306
Kappa between actual labels and annotator 2: 0.17821782178217827


In [16]:
# precision, recall и F1-score для каждого разметчика относительно фактических меток

# Рассчитываем метрики для разметчика 1
precision_1 = precision_score(result_df['AI_notAI'], result_df['label_1binary'])
recall_1 = recall_score(result_df['AI_notAI'], result_df['label_1binary'])
f1_1 = f1_score(result_df['AI_notAI'], result_df['label_1binary'])

# Рассчитываем метрики для разметчика 2
precision_2 = precision_score(result_df['AI_notAI'], result_df['label_2binary'])
recall_2 = recall_score(result_df['AI_notAI'], result_df['label_2binary'])
f1_2 = f1_score(result_df['AI_notAI'], result_df['label_2binary'])

print("Metrics for Annotator 1:")
print(f"Precision: {precision_1}")
print(f"Recall: {recall_1}")
print(f"F1-score: {f1_1}")

print("\nMetrics for Annotator 2:")
print(f"Precision: {precision_2}")
print(f"Recall: {recall_2}")
print(f"F1-score: {f1_2}")

Metrics for Annotator 1:
Precision: 0.9607843137254902
Recall: 0.9702970297029703
F1-score: 0.9655172413793103

Metrics for Annotator 2:
Precision: 0.6323529411764706
Recall: 0.42574257425742573
F1-score: 0.5088757396449705


### Согласованность между разметчиками:
Коэффициент Каппа Коэна между разметчиками 1 и 2 составляет 0.21, это может быть интерпретировано как слабая согласованность.

### Согласованность с фактическими метками:
Коэффициент Каппа Коэна между фактическими метками и разметкой от разметчика 1 равен 0.93. Это высокое значение указывает на высокую согласованность между разметкой разметчика 1 и фактическими метками.

Коэффициент Каппа Коэна между фактическими метками и разметкой от разметчика 2 равен 0.18. Это значение указывает на слабую согласованность между разметкой разметчика 2 и фактическими метками.

### Квалификация разметчиков (Метрики):
Для разметчика 1 метрики Precision, Recall и F1-score показывают высокие значения, что указывает на высокую точность и полноту его разметки.

Для разметчика 2 метрики Precision и Recall ниже, что может свидетельствовать о менее качественной разметке.

### Выводы:
Между разметчиками существует слабая согласованность, что может быть связано с различиями в их методах разметки.

Разметчик 1 демонстрирует высокую согласованность с фактическими метками, в то время как разметчик 2 показывает более низкую согласованность.

Разметчик 1 обладает более высокой квалификацией, поскольку его метрики ближе к идеальным значениям.

## Обратная связь по разметке данных:

Понятность ТЗ оба разметчика оценили на 5/5

- Сложно ли было размечать изображения?

Разметчик 1: Нет.
Разметчик 2: Сложно, потому что изображения очень похожи.

- Понятен ли интерфейс платформы для разметки?

Разметчик 1: Все было понятно.
Разметчик 2: Интерфейс понятен и удобен.

- Что можно улучшить?

Разметчик 1: Добавить кол-во пройденных этапов (изображений).
Разметчик 2: Сделать выбор не из выпадающего списка, а из 2 вариантов (например, поставить галочку).

### Общие выводы:

Разметчики воспринимают интерфейс платформы как понятный, что является положительным аспектом.
Схожесть изображений может представлять вызов, поэтому дополнительные меры, такие как учет пройденных этапов, могут сделать процесс более прозрачным и управляемым.